In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\student-performance-prediction\\experiment'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Lenovo\\Desktop\\student-performance-prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [7]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [8]:
## Data Ingestion Component

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    # download the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following ingo: \n{headers}")
        else:
            logger.info(f"file already exists")
            
    # extract the zip file
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-02 09:06:36,709: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-02 09:06:36,717: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-02 09:06:36,735: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-02 09:06:36,736: INFO: common: created directory at: artifacts]


{'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/krishnaik06/datasets/raw/refs/heads/main/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}
[2025-12-02 09:06:36,736: INFO: common: created directory at: artifacts/data_ingestion]
[2025-12-02 09:06:36,736: INFO: 1196497530: file already exists]
